# PyNAS

This notebook describes how to use NAS for generating and evolving neural network models.

In [ ]:
%load_ext autoreload
%autoreload 2

Test if GPU is available (I am running on mac..)

In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU count:", torch.cuda.device_count())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

## 🧠 PyTorch Lightning Setup Script

This script initializes a training environment for a neural network using PyTorch Lightning and a custom NAS framework.

🔍 **Explanation**

- **Dataset Loader:** Uses RawClassifierDataModule to load data with batch_size=4, num_workers=2, and no transforms.
- **Config Parsing:** Reads logs_dir_GA and seed from config.ini.
- **Environment Setup:**  
	- Enables full column display in pandas.  
	- Ensures reproducibility with pl.seed_everything().  
	- Sets matrix multiplication precision to "medium" for optimized performance.



In [ ]:
import configparser
import pandas as pd
import sys 
sys.path.append('..')
import torch
import pytorch_lightning as pl

from pynas.core.population import Population
from scripts.dataloader import SegmentationDataModule

# Define dataset module
root_dir = './data'
dm = SegmentationDataModule(root_dir, batch_size=8, num_workers=6, transform=None)

config = configparser.ConfigParser()
config.read('./src/pynas/core/config.ini')
def setting():
    pd.set_option('display.max_colwidth', None)
    # Logging
    logs_directory = str(config['GA']['logs_dir_GA'])
    # Torch stuff
    seed = config.getint(section='Computation', option='seed')
    pl.seed_everything(seed=seed, workers=True)  # For reproducibility
    torch.set_float32_matmul_precision("medium")  # to make lightning happy
setting()

## ⚙️ Genetic Algorithm Model Setup

This section of code configures parameters for a Genetic Algorithm (GA)-based Neural Architecture Search (NAS) using the Population class.

In [ ]:
config = configparser.ConfigParser()
config.read('./src/pynas/core/config.ini')

print("Configuration loaded from:", config)
# Model parameters
max_layers = 7 # Maximum number of layers in the model: composed of normal cell (Conv Block) and reduction cell (i.e. Pooling layer)
max_iter = int(config['GA']['max_iterations'])
# GA parameters
n_individuals = int(config['GA']['population_size'])
k_best = int(config['GA']['k_best'])
n_random = int(config['GA']['n_random'])
mating_pool_cutoff = float(config['GA']['mating_pool_cutoff'])
mutation_probability = float(config['GA']['mutation_probability'])

pop = Population(n_individuals=n_individuals, max_layers=max_layers, dm=dm, max_parameters=400_000)

### 🧬 Initial Population (`Population` Object)

The initial population defines the starting point for the Genetic Algorithm (GA)-based search. It consists of a set of randomly generated neural architectures (individuals), each encoded with:

- A **variable number of layers** (up to `max_layers`)
- **Random hyperparameters and layer types** within predefined search constraints
- A **bounded total parameter count** (`max_parameters`) to limit model complexity


In [ ]:
pop.initial_poll()

### 🔄 `train()` vs `evolve()` in a GA-based NAS Framework

In the context of the `Population` class within a Genetic Algorithm (GA)-driven Neural Architecture Search (NAS), the methods `train()` and `evolve()` serve distinct purposes in the evolutionary pipeline.

---

### 🏋️‍♂️ `train()`

Trains all individuals (i.e., neural architectures) in the current population.

#### 📌 Responsibilities:
- Performs forward and backward passes on the dataset (`dm`).
- Optimizes model weights using a standard training loop.
- Evaluates performance (fitness), typically via validation accuracy or loss.
- Stores fitness scores used for selection in the GA.

#### ✅ Outcome:
Each individual's **fitness value** is updated and can now be ranked for survival and reproduction.

---

### 🧬 `evolve()`

Applies evolutionary operations to produce the **next generation** of architectures.

#### 📌 Responsibilities:
- **Selection**: Ranks individuals by fitness and selects top performers (according to `mating_pool_cutoff`).
- **Crossover**: Combines architecture components (e.g., layer types, connections) from parent individuals.
- **Mutation**: Applies random changes (with `mutation_probability`) to maintain diversity.
- **Population Replacement**: Creates a new generation of individuals.

#### ✅ Outcome:
A **new population** is generated with architectural variations derived from the most promising candidates of the previous generation.

In [ ]:
epochs = int(config['GA']['epochs'])
for _ in range(max_iter):
    pop.train_generation(task='segmentation', lr=0.001, epochs=epochs, batch_size=8) 
    pop.evolve(mating_pool_cutoff=mating_pool_cutoff, mutation_probability=mutation_probability, k_best=k_best, n_random=n_random)

### 📄 `load_dataframe()` Method – What It Does

The `load_dataframe` method in the `Population` class is used to retrieve 📦 **stored results or evaluation metrics** from the training and evolution process of the models.

When calling `pop.load_dataframe(9)`, it loads data such as:
- 📊 **Performance metrics**
- 📉 **Loss values**
- 🧠 **Architectural configurations**

These were saved during the evolutionary search.

You can use this data for:
- 🔍 **Analysis** of model performance
- 📈 **Visualization** of evolutionary dynamics
- 🛠️ **Further processing** like re-training or model selection

⚠️ **Note**: The index passed (e.g., `9`) must correspond to the specific generation or result set you want to inspect.

# Inference

Using the evaluated and saved model. We use the traced pytroch model (.pt) to load and execute inference.

In [ ]:
# Load the saved TorchScript model and test with a dummy input.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

save_path = "models_traced/generation_9/model_and_architecture_19.pt"
loaded_model = torch.jit.load(save_path, map_location=device)
loaded_model.eval()

# Ensure input is moved to the correct device
example_input = torch.randn(1, *dm.input_shape).to(device)
example_input = example_input.to(device)

with torch.no_grad():
    output = loaded_model(example_input)
print("Output from the loaded model:", output)